In [1]:
import xarray as xr
import numpy as np
import geopy.distance as gpd

In [3]:
Buoy = 'E05'
springtimeE05 = xr.open_dataset("LiDAR/springtime_"+Buoy+".nc")
#jet_idE05 = xr.open_dataset('LiDAR/springtime_'+Buoy+'_LLJs_2020.nc')

In [4]:
Buoy = 'E06'
springtimeE06 = xr.open_dataset("LiDAR/springtime_"+Buoy+".nc")
jet_idE06 = xr.open_dataset('LiDAR/springtime_'+Buoy+'_LLJs_2020.nc')

In [5]:
cases = dict({
    'E05': dict({
        '20200602': [8928, 9792],   # 2020-06-02 to 2020-06-08
        '20200513': [6048, 6912],   # 2020-05-13 to 2020-05-19
        '20200403': [288, 1152]     # 2020-04-03 to 2020-04-09
    }),
    'E06': dict({
        '20200602': [8347, 9211],
        '20200513': [5467, 6331],
        '20200403': [288, 1152],    # 2020-04-03 to 2020-04-09
        # '20200405': [576, 1008], 
        # '20200429': [4032, 4200], 
        # '20200514': [5616, 6192], 
        # '20200521': [6624, 6912], 
        # '20200603': [8496, 9072], 
        # '20200618': [10656, 10800],
        # '20200628': [12096, 12240]
    })
})

In [29]:
case = '20200403'
t_start = cases['E05'][case][0] + 2*24*6 + 12
t_end = t_start + 24*6
e05 = springtimeE05.isel(t=slice(t_start, t_end))
t_start = cases['E06'][case][0] + 2*24*6 + 12
t_end = t_start + 24*6
e06 = springtimeE06.isel(t=slice(t_start, t_end))
e06

<xarray.Dataset>
Dimensions:        (t: 144, z: 10)
Coordinates:
    datetime       (t) datetime64[ns] ...
    height         (z) int64 18 38 58 78 98 118 138 158 178 198
Dimensions without coordinates: t, z
Data variables:
    windspeed      (t, z) float64 ...
    winddirection  (t, z) float64 ...
    temp           (t) float64 ...
    pressure       (t) float64 ...
    rel_humidity   (t) float64 ...
    waterT         (t) float64 ...
    U              (t, z) float64 ...
    V              (t, z) float64 ...

In [30]:
dU = e05['U'] - e06['U']
dV = e05['V'] - e06['V']
dS = e05['windspeed'] - e06['windspeed']

e05latlon = (39+58/60+9.4/3600, 72+43/60+.09/3600)
e06latlon = (39+32/60+48.38/3600, 73+25/60+44.01/3600)

In [31]:
dV

<xarray.DataArray 'V' (t: 144, z: 10)>
array([[-1.49497194, -1.59234998, -1.61475292, ..., -1.5223584 ,
        -1.55220063, -1.61736613],
       [-0.73297898, -0.66484795, -0.99100842, ..., -0.56466668,
        -0.45055915, -0.47666607],
       [-1.11004143, -1.06297895, -1.0618773 , ..., -0.50340095,
        -0.43466065, -0.42452834],
       ...,
       [ 0.85310653,  0.86206831,  1.05184291, ...,  1.06077877,
         1.29480424,  1.24644932],
       [ 0.69548106,  0.81298188,  0.73725675, ...,  0.8357829 ,
         1.05948832,  0.83626919],
       [ 0.54088433,  0.60113834,  0.50026696, ...,  1.78791024,
         1.86632647,  1.48467605]])
Coordinates:
    datetime  (t) datetime64[ns] 2020-04-05T02:00:00 ... 2020-04-06T01:50:00
    height    (z) int64 18 38 58 78 98 118 138 158 178 198
Dimensions without coordinates: t, z

In [32]:
d = gpd.geodesic(e05latlon, e06latlon).m
dx = gpd.geodesic(e05latlon, (e05latlon[0], e06latlon[1])).m
dy = gpd.geodesic(e05latlon, (e06latlon[0], e05latlon[1])).m

In [33]:
print(d, dx, dy)

76976.68700978138 60844.590238182755 46910.78639130137


In [34]:
dUdx = dU.mean(axis=0)/dx
dVdy = dV.mean(axis=0)/dy
dSdd = dS.mean(axis=0)/d

In [35]:
dUdy = dU.mean(axis=0) / dy
dVdx = dV.mean(axis=0) / dx
curl = dVdx - dUdy
print(curl)
print(curl.mean())

<xarray.DataArray (z: 10)>
array([4.16669211e-05, 4.58232374e-05, 4.98423494e-05, 5.17905291e-05,
       5.15906778e-05, 4.60920064e-05, 3.87152419e-05, 3.68459886e-05,
       3.73267901e-05, 4.00842668e-05])
Coordinates:
    height   (z) int64 18 38 58 78 98 118 138 158 178 198
Dimensions without coordinates: z
<xarray.DataArray ()>
array(4.39778009e-05)


In [70]:
# 2020-04-03 full period: curl = 3.77298935e-05
# 2020-04-03 jet only: curl = 4.39778009e-05

# 2020-05-15 jet only: curl = -3.36615836e-05

# 2020-06-02 full period: curl = -2.24095403e-05
# 2020-06-02 one day: curl = 3.63755529e-05
# 2020-06-03 one day: curl = -6.46008676e-05
# 2020-06-04 one day: curl = -2.04727118e-05
# 2020-06-05 one day: curl = -4.54571357e-05

## From Zhang (2006) ref to Blackadar
$$ F = \sqrt{f(f+dV/ds)}$$

In [36]:
lat = 39.68 # E06 NYSERDA
f = 2*7.2921e-5 * np.sin(lat/180 * np.pi) # coriolis term
T = 27.0
F = 2*np.pi/T/3600 # approx for 25h period

In [37]:
shear = F**2/f - F
f_tilde = np.sqrt(f*(f+curl.mean()))
T_tilde = 2*np.pi/f_tilde/3600
print(shear, f, F)
print(f_tilde, T_tilde)

-1.9768914295320393e-05 9.312000141950217e-05 6.464182414793813e-05
<xarray.DataArray ()>
array(0.00011299) <xarray.DataArray ()>
array(15.44687507)


In [ ]:
# 2020-04-03 jet only: f = 15.44687507
# 2020-05-15 jet only: f = 23.45574364